# Homework 4

Gary Hoppenworth

Sources: 
* [Floyd's cycle algorithm](https://en.wikipedia.org/wiki/Cycle_detection),
* [Pollard's $\rho$ algorithm](https://en.wikipedia.org/wiki/Pollard%27s_rho_algorithm),
* [multiplicative order](https://en.wikipedia.org/wiki/Multiplicative_order),
* [multiplicative groups of integers modulo N](https://en.wikipedia.org/wiki/Multiplicative_group_of_integers_modulo_n)
* *A First Course in Abstract Algebra-- JB Fraleigh*
* Quantum Computing Slides -- Professor Wocjan

# Problem 1: Implementing Floyd's cycle finding algorithm

The idea of Floyd's cycle finding algorithm is that if there is a cycle in a function $f$ that occurs after some point $x_0$, then there must be some $x$ such that $f^{x}(x_0) = f^{2x}(x_0)$, where $f^x$ denotes the composition of $f$ $x$ times. Once this $x$ is found, the algorithm uses some tricks to find the start of the cycle $\mu$ and the length of the cycle $\lambda$.  


Note: The following implementation is based on the code presented in the Cycle Detection Algorithms wikipedia page (it is essentially the same as the code presented there, but I wrote it myself if that matters).

In [0]:
def floyds_alg(x0, f):

  slow = f(x0)
  fast = f(f(x0))

  #check if f^x(x0) = f^{2x}(x0) starting with x = 1, until true
  while slow != fast:
    slow = f(slow)
    fast = f(f(fast))

  #Now the idea is that the slow pointer and fast pointer are \gnu iterations 
  #apart. The cycle length MUST be a factor of \gnu. Then if we start the slow
  #pointer at x0 and have the slow and fast pointer iterate through f at the
  #same time, then the first value where slow == fast will be the first point
  #in the cycle, since we know that once we start the cycle, slow will equal 
  #fast for the remaining iterations

  slow = x0
  while slow != fast:
    slow = f(slow)
    fast = f(fast)


  #We now know that the start of the loop is the current location of slow
  mu = slow

  #Now we have the starting point \mu of the cycle.
  #To find the true length of the cycle (not just a factor of it like \mu),
  #we simply start at \mu and iterate until we reach \mu again.

  lambda_val = 1
  slow = f(slow)

  while slow != mu:
    slow = f(slow)
    lambda_val += 1



  return mu, lambda_val

## Testing my implementation

### Test 1

I define a function $f: \{1, 2, 3, 4, 5, 6\} \rightarrow \{1, 2, 3, 4, 5, 6\}$ as follows:

* $f(1) = 2$
* $f(2) = 3$
* $f(3) = 4$
* $f(4) = 5$
* $f(5) = 6$
* $f(6) = 4$

Observe that this function contains a cycle that starts at $4$ and has length $3$.

In [2]:
#the function $f$
def f(x):
  if x == 1:
    return 2
  elif x == 2:
    return 3
  elif x == 3:
    return 4
  elif x == 4:
    return 5
  elif x == 5:
    return 6
  elif x == 6:
    return 4

#initial x
x0 = 1

mu, lambda_val = floyds_alg(x0, f)
print("The cycle in f starts at ", mu)
print("The cycle in f has length ", lambda_val)

The cycle in f starts at  4
The cycle in f has length  3


`floyds_alg` worked as expected!

### Test 2

Now I devise a function $f(x)$ as follows. Let $g(x)$ be a psuedorandom integer generator that takes a seed $x$ and outputs a 'random' integer between $1$ and $1000$. Note that $g$ is well-defined, since $g$ will have the same output on the same seed. Now we define our function $f(x)$ as
$$f(x) = g(x) \mbox{ mod } 111$$.

Now since $f(x)$ can only take on $111$ unique values, there will be some $i \neq j$ such that $f^i(x) = f^j(x)$. Then $f$ will contain a loop, since each value of $f$ depends only on its input $x$. This function $f(x)$ was inspired by a similar function used in Pollard's $\rho$ algorithm.


In [3]:
from random import seed, randint

#   our function $g(x)$ that takes a seed x, and returns the 'random' number between
#   1 and 1000 corresponding to that seed
def g(x):
  seed(x)
  return randint(1, 1000)


#   our function f that on input x, returns g(x) modulo 20.
#   we will print f(out) at each step so we can verify that 
#   `floyds_alg` is behaving correctly.

def f(x):
  g_out = g(x)
  f_out = g_out % 111
  return f_out


#CHANGE THIS VALUE TO TRY A NEW ITERATION SEQUENCE
x0 = 2

mu, lambda_val = floyds_alg(x0, f)


#We now print out the values of $f$ to verify that `floyds_alg` is working
x = x0
cycle_started = False

#we iteratively compute and print f(x) until f(x) = \mu, and the cycle has started
while not cycle_started:
    if x == mu:
      cycle_started = True
    f_out = f(x)
    print("f(" + str(x) + ") \t = \t " + str(f_out))
    x = f_out

#then we iteratively print more values of our iteration until we have performed
#one loop
for i in range(lambda_val):
    f_out = f(x)
    print("f(" + str(x) + ") \t = \t " + str(f_out))
    x = f_out

print("\nThe cycle starts at " + str(mu)  + ", and the cycle has length " + str(lambda_val))

f(2) 	 = 	 91
f(91) 	 = 	 86
f(86) 	 = 	 34
f(34) 	 = 	 98
f(98) 	 = 	 32
f(32) 	 = 	 80
f(80) 	 = 	 57
f(57) 	 = 	 44
f(44) 	 = 	 86
f(86) 	 = 	 34

The cycle starts at 86, and the cycle has length 7


So far I have tested `floyds_alg` on inputs `x0 = 2, 24, 96` and it appears to work on function $f$. The average number of iterations before $f$ enters a loop seems surprisingly low, considering that $f$ takes on $111$ different values. Perhaps this has to do with how the random seed works.

**You may experiment by trying different values of `x0` or even changing the modulo number from $111$**.

# Problem 2: implementing `find_order`
 





We say that the (multiplicative) order $r$ of   $a\mbox{ mod }N$ is the the smallest positive integer $r$ such that $a^r \equiv 1 \mbox{ mod } N$.

**NOTE: This algorithm assumes that $a$ and $N$ are coprime. This is because if $a$ and $N$ are not coprime, then the multiplicative order of $a \mbox{ mod }N$ does not exist. This can be seen because in the ring of integers modulo $N$, the
 elements $x$ that are coprime with $N$ are exactly the elements of the ring with a multiplicative inverse ([source](https://en.wikipedia.org/wiki/Multiplicative_group_of_integers_modulo_n)).**

By our assumption that $a$ is coprime to $N$, we have that for all $x \in \mathbb{N}$, the congruence class $a^x \mbox{ mod } N$ corresponds to an element of the multiplicative group of integers modulo $N$ (written as $(\mathbb{Z} / N \mathbb{Z})^{\times}$). We will use this fact later.


To use Floyd's cycle finding algorithm to find the order $r$, we define the function 

$$f(x) = ax \mbox{ mod } N$$

Note that $f(1) = a \mbox{ mod } N$. Now suppose that $f^x(1) = a \mbox{ mod } N$ for some $x > 1$. Then we have

$$
f(1) = f^x(1) \\
a \mbox{ mod } N = a^x \mbox{ mod } N \\
\mbox{Then by the cancellation property of groups...}\\
aa^{-1} \mbox{ mod } N = a^xa^{-1} \mbox{ mod } N \\
1 \mbox{ mod } N = a^{x-1} \mbox{ mod } N \\
$$

So if $x$ is the least $x>1$ such that $f^x(1) = a \mbox{ mod } N$, then $x-1$ is the least positive integer such that $f^{x-1}(1) = 1 \mbox{ mod } N$. It follows that $ r = x-1$.

In other words, we simply need to find the length $\lambda$ of the cycle in function $f(x)$ in order to find our multiplicative order $r$.


Then we can use `floyds_alg` with inputs $x_0 = 1$ and function $f(x)$ to find  $\lambda$ and thus find $r$!

In [0]:
def find_order(a, N):

  #define our function f
  f = lambda x : (a * x) % N

  #define our starting point x0
  x0 = 1

  #compute lambda, the length of the cycle
  _, lambda_val = floyds_alg(x0, f)

  return lambda_val


## Testing my `find_order` function

###Test 1:
 I test the following pairs of $a, N$:

* $a = 1, N = 5$: then $a \mbox{ mod } N$ has order $1$.
* $a = 2, N = 7$: then $a \mbox{ mod } N$ has order $3$, since $2^3 = 8 = 1 \mbox{ mod }7$.



In [5]:
order = find_order(1, 5)
print("1 mod 5 has order " + str(order))

order = find_order(2, 7)
print("2 mod 7 has order " + str(order))


1 mod 5 has order 1
2 mod 7 has order 3


Tests successful!

### Test 2:
Now I will make use of Fermat's Little Theorem, which states that for a prime $p$ and any $a$ such that $a \mbox{ mod } P \not \equiv 0$,  we have that $a^{p-1} \equiv 1 \mbox{ mod } P$. **Then the true order of $a$ must divide $p-1$.** I will use this to test my order function on large $p$ and $a$.

### I will test:

* $a = 2020, N = 2017$ (Note that $2017$ is prime and $2020 \neq 0 \mbox{ mod }N$)
* $a = 13, 666, N = 2027$ (Note that $2027$ is prime and neither $13$ nor $666$ equal $0 \mbox{ mod } N$) 

In [6]:
order = find_order(2020, 2017)
print("2020 mod 2017 has order " + str(order) + "\n")
print("Order " + str(order) + " divides p - 1 = 2016:\n " + str(2016 % order == 0))

2020 mod 2017 has order 1008

Order 1008 divides p - 1 = 2016:
 True


The order of $2020 \mbox{ mod } 2017$ divides $2016$ as expected!

In [7]:
order = find_order(13, 2027)
print("13 mod 2027 has order " + str(order) + "\n")
print("Order " + str(order) + " divides p - 1 = 2026:\n " + str(2026 % order == 0) + "\n\n\n")

order = find_order(666, 2027)
print("666 mod 2027 has order " + str(order) + "\n")
print("Order " + str(order) + " divides p - 1 = 2026:\n " + str(2026 % order == 0))

13 mod 2027 has order 1013

Order 1013 divides p - 1 = 2026:
 True



666 mod 2027 has order 1013

Order 1013 divides p - 1 = 2026:
 True


The order of $13 \mbox{ mod } 2027$ divides $2026$ as expected! 

Likewise, the order of $666 \mbox{ mod } 2027$ divides $2026$ as expected!

# Problem 3: implementing `factor`
Here I implement the `factor` function which finds a nontrivial factor of an input $N$. This function makes use of Miller's algorithm for integer factorization that makes use of order finding in the multiplicative group of integers modulo $N$.

**Assumptions:**

In this homework problem I assume that $N$ is not prime and that $N$ contains at least two distinct prime factors. This is a reasonable assumption because it can be efficiently checked if a number is prime, or if a number is the power of a prime. I do not know how to implement the algorithms that solve these problems efficiently, so I simply assume $N$ has our desired properties.


In [0]:
from math import gcd
from random import randint

def factor(N):

  #If $N$ is even, return 2. Now we may assume $N$ is odd.
  if N % 2 == 0:
    return 2

  while True:
    #pick random integer between 2 and N - 1
    a = randint(2, N -1)

    #if GCD(a, N) is not 1, then $a$ is a factor of $N$
    if gcd(a, N) != 1:
      return a
    
    else:
      #we compute the order using `find_order`
      r = find_order(a, N)

      #if the order is divisible by two
      if r % 2 == 0:

        #   Miller shows that if $a \mod N$ has order $r$, then
        #   $(a^{r/2} - 1)$ may have a nontrivial factor that is also
        #   a factor of $N$, since $(a^{r/2} - 1)(a^{r/2} + 1) = 0 \mod N$
        #   we can find this factor if it exists by computing the gcd of
        #   $(a^{r/2} - 1)$ and $N$

        f = gcd(a ** (r // 2) - 1, N)

        #if $f$ is not trivial
        if f != 1 and f != N:
          return f



## Testing my `factor` function:

### Test 1:
I test the following values of $N$:

* $N = 323 = 17 * 19$
* $N =  17947 = 131 * 137$, ($131$ and $137$ are prime)

In [9]:
#N = 323
f = factor(323)
print("323 has nontrivial factor " + str(f) + "\n\n")

#N = 17947
f = factor(17947)
print("17947 has nontrivial factor " + str(f))

323 has nontrivial factor 17


17947 has nontrivial factor 131


`factor` produced correct factors of our numbers $n$!

### Test 2:
I now try the $8$th Fermat number, $F_8$!

$F_8 = 2^{2^8} + 1$

Note that $F_8$ has factors $1238926361552897$ and  $93461639715357977769163558199606896584051237541638188580280321$.

In [10]:
N = 2 ** (2 ** 8) + 1
f = factor(N)
print("The 8th Fermat Number has nontrivial factor " + str(f))

KeyboardInterrupt: ignored

I let `factor` run for about an hour on $F_8$ before I gave up.

 The $8$th Fermat number was first factorized by Pollard's $\rho$ algorithm in $1981$. It took $2$ hours for this algorithm to run on their computing system.